In [ ]:
import os
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score


In [ ]:
#model files i.e. in vorpal/models
clf = joblib.load('')
feature_vector = pd.read_pickle('')
#training labels
meta = pd.read_table('')
#test labels
test_meta = pd.read_table('')

In [ ]:
#training
os.chdir('') #directory for bed files produced by referencemapping_mp.py
bed_files = [file for file in os.listdir() if '.bed' in file]

beds = [pd.read_table(f,names=['chr','start','end','name','score']) for f in bed_files]

#parsing Seq ID step required for RVDB formatted fasta headerrs
for i in range(len(beds)):
    beds[i]['chr'] = [a[2] for a in beds[i]['chr'].str.split('|').values]


feature_dict = {}

print("Building feature dict.")
for b in beds:
    accession_num = b['chr'].unique()[0]
    feature_dict[accession_num]={f:0 for f in feature_vector}
    feature_dict[accession_num].update(b['name'].value_counts().to_dict())

feature_table = pd.DataFrame(feature_dict).T.fillna(0.0)
feature_table.index.name = "accession"
feature_table.reset_index(inplace=True)
training_table = pd.merge(feature_table,meta)
print("Dropping ambiguous labels.")
training_table = training_table[training_table['label'] != -1]
train_labels = training_table['label']

In [ ]:
#accuracy score on training data
train_features = training_table[feature_vector].copy()
accuracy_score(train_labels,clf.predict(train_features.values))

In [ ]:
#misclassifieds out
training_table['predict'] = clf.predict(train_features.values)
training_table['predict_proba'] = clf.predict_proba(train_features.values)[:,1]
training_table[training_table['label'] != training_table['predict']][['accession','label','groups','predict','predict_proba']].to_csv('',sep='\t',index=False)

In [ ]:
#test
os.chdir('') #directory for bed files produced by referencemapping_mp.py
bed_files = [file for file in os.listdir() if '.bed' in file]

beds = [pd.read_table(f,names=['chr','start','end','name','score']) for f in bed_files]

feature_dict = {}

print("Building feature dict.")
for b in beds:
    accession_num = b['chr'].unique()[0]
    feature_dict[accession_num]={f:0 for f in feature_vector}
    feature_dict[accession_num].update(b['name'].value_counts().to_dict())

feature_table = pd.DataFrame(feature_dict).T.fillna(0.0)
feature_table.index.name = "accession"
feature_table.reset_index(inplace=True)
test_table = pd.merge(feature_table,test_meta)
print("Dropping ambiguous labels.")
test_table = test_table[test_table['label'] != -1]
test_labels = test_table['label']

In [ ]:
#accuracy score on test data
test_features = test_table[feature_vector].copy()
accuracy_score(test_labels,clf.predict(test_features.values))

In [ ]:
#test set probabilities out
test_table['predict'] = clf.predict(test_features.values)
test_table['predict_proba'] = clf.predict_proba(test_features.values)[:,1]
test_table[['accession','label','predict','predict_proba']].to_csv('',sep='\t',index=False)

In [ ]:
#predict on new sequence
os.chdir('') #directory for bed files produced by referencemapping_mp.py
bed_files = [file for file in os.listdir() if '.bed' in file]

beds = [pd.read_table(f,names=['chr','start','end','name','score']) for f in bed_files]

feature_dict = {}

print("Building feature dict.")
for b in beds:
    accession_num = b['chr'].unique()[0]
    feature_dict[accession_num]={f:0 for f in feature_vector}
    feature_dict[accession_num].update(b['name'].value_counts().to_dict())

predict_table = pd.DataFrame(feature_dict).T.fillna(0.0)
predict_table.index.name = "accession"
predict_table.reset_index(inplace=True)

In [ ]:
#show class predictions and probabilities for new sequences
predict_features = predict_table[feature_vector].copy()
predict_table['predict'] = clf.predict(predict_features.values)
predict_table['predict_proba'] = clf.predict_proba(predict_features.values)[:,1]
predict_table[['accession','predict','predict_proba']]